In [23]:
import pandas as pd
import numpy as np

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [25]:
df = pd.read_csv('train.csv')

In [26]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [28]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['Survived'], axis=1), 
    df['Survived'], 
    test_size=0.2, 
    random_state=42)

In [30]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [31]:
y_train.sample(5)

273    0
226    1
113    0
271    1
37     0
Name: Survived, dtype: int64

In [32]:
# Imputation transformer

tnf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
    ], remainder='passthrough')

In [33]:
# OneHotEncoding transformer

tnf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), [1,6])
], remainder='passthrough')

In [34]:
# Scaling Transformer

tnf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,7))
])

In [35]:
# Feature Selection

tnf4 = SelectKBest(score_func=chi2, k=5)

In [36]:
# Train the model
tnf5 = DecisionTreeClassifier()

### Create Pipeline

In [37]:
pipe = Pipeline([
    ('tnf1', tnf1),
    ('tnf2', tnf2),
    ('tnf3', tnf3),
    ('tnf4', tnf4),
    ('tnf5', tnf5)
])

### Pipeline VS make_pipeline

Pipeline requires naming of steps, make_pipeline does not

same applies to ColumnTransformer vs Make_column_transformer

In [38]:
# pipe = make_pipeline(tnf1, tnf2, tnf3, tnf4, tnf5)

In [39]:
pipe.fit(X_train, y_train)

c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 7, None))])),
                ('tnf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002566E7B9440>)),
                ('tnf5', DecisionTreeClassifier())])

In [40]:
y_pred = pipe.predict(X_test)

c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### Explore Pipeline

In [41]:
pipe.named_steps

{'tnf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tnf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tnf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 7, None))]),
 'tnf4': SelectKBest(k=5, score_func=<function chi2 at 0x000002566E7B9440>),
 'tnf5': DecisionTreeClassifier()}

In [42]:
pipe.named_steps['tnf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [43]:
pipe.named_steps['tnf1'].transformers_[0][1].statistics_

array([29.49884615])

In [44]:
pipe.named_steps['tnf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [45]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5865921787709497

### Cross Validation using Pipeline

In [48]:
# Cross Validation using cross_val_score

from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, scoring='accuracy').mean()

c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\U

0.6068354181030238

### GridSearch using Pipeline

In [52]:
# GridSearchCV

params = {
    'tnf5__max_depth' : [1,2,3,4,5,None]
}

In [53]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\SUNIL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\U

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tnf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('tnf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('tnf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 7, None))])),
                                       ('tnf4',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x000002566E7B9440>)),
                                       ('tnf5', DecisionTreeClassifier())]),
             param_grid={'tnf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [54]:
grid.best_score_

0.6068354181030238

In [55]:
grid.best_params_

{'tnf5__max_depth': 2}

In [56]:
grid.best_estimator_

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 7, None))])),
                ('tnf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002566E7B9440>)),
                ('tnf5', DecisionTreeClassifier(max_depth=2))])

### Exporting Pipeline

In [57]:
import pickle
pickle.dump(pipe, open('models/pipe.pkl', 'wb'))